Thank you so much for sticking with this learning module until the end. You learned a lot, and now you have the basic tools to make predictions using your own data. You learned how to prepare your data, how to define your neural network, how to train and test it, and finally how to make a prediction using your trained network. 

In this notebook, we'll bring together the most important parts of the code you saw throughout this module, so you can easily refer to it in the future.

In [1]:
import gzip
import numpy as np
import tensorflow as tf
from typing import Tuple


class NeuralNetwork(tf.keras.Model):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.sequence = tf.keras.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(512),
      tf.keras.layers.ReLU(),
      tf.keras.layers.Dense(512),
      tf.keras.layers.ReLU(),
      tf.keras.layers.Dense(10)
    ])

  def call(self, x: tf.Tensor) -> tf.Tensor:
    y_prime = self.sequence(x)
    return y_prime


labels_map = {
    0: 'T-Shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle Boot',
  }


def read_images(path: str, image_size: int, num_items: int) -> np.ndarray:
  f = gzip.open(path,'r')
  buf = f.read(image_size * image_size * num_items)
  data = np.frombuffer(buf, dtype=np.uint8)
  data = data.reshape(num_items, image_size, image_size)
  return data


def read_labels(path: str, num_items: int) -> np.ndarray:
  f = gzip.open(path,'r')
  f.read(8)
  buf = f.read(num_items)
  data = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
  data = data.reshape(num_items)
  return data


def get_data(batch_size: int) -> Tuple[tf.data.Dataset, tf.data.Dataset]:
  image_size = 28
  num_train = 60000
  num_test = 10000

  training_images = read_images('data/FashionMNIST/raw/train-images-idx3-ubyte.gz', image_size, num_train)
  test_images = read_images('data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz', image_size, num_test)
  training_labels = read_labels('data/FashionMNIST/raw/train-labels-idx1-ubyte.gz', num_train)
  test_labels = read_labels('data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz', num_test)

  # (training_images, training_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

  train_dataset = tf.data.Dataset.from_tensor_slices((training_images, training_labels))
  test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

  train_dataset = train_dataset.map(lambda image, label: (float(image) / 255.0, label))
  test_dataset = test_dataset.map(lambda image, label: (float(image) / 255.0, label))

  train_dataset = train_dataset.batch(batch_size).shuffle(500)
  test_dataset = test_dataset.batch(batch_size).shuffle(500)

  return (train_dataset, test_dataset)


def training_phase():
  learning_rate = 0.1
  batch_size = 64
  epochs = 2

  (train_dataset, test_dataset) = get_data(batch_size)

  model = NeuralNetwork()

  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  optimizer = tf.keras.optimizers.SGD(learning_rate)
  metrics = ['accuracy']
  model.compile(optimizer, loss_fn, metrics)

  print('\nFitting:')
  model.fit(train_dataset, epochs=epochs)
    
  print('\nEvaluating:')
  (test_loss, test_accuracy) = model.evaluate(test_dataset)
  print(f'\nTest accuracy: {test_accuracy * 100:>0.1f}%, test loss: {test_loss:>8f}')

  model.save_weights('outputs/weights')


def inference_phase():
  batch_size = 64

  model = NeuralNetwork()
  model.load_weights('outputs/weights').expect_partial()

  (_, test_dataset) = get_data(batch_size)
  (X_batch, actual_index_batch) = test_dataset.as_numpy_iterator().next()
  X = X_batch[0:3, :, :]
  actual_indices = actual_index_batch[0:3]

  predicted_indices = model.predict(X)

  print('\nPredicting:')
  for (actual_index, predicted_index) in zip(actual_indices, predicted_indices):
    actual_name = labels_map[actual_index]
    predicted_name = labels_map[tf.math.argmax(predicted_index).numpy()]
    print(f'Actual: {actual_name}, Predicted: {predicted_name}')


training_phase()
inference_phase()



Fitting:
Epoch 1/2
938/938 [==============================] - 7s 6ms/step - loss: 0.5567 - accuracy: 0.8024
Epoch 2/2
938/938 [==============================] - 6s 6ms/step - loss: 0.3958 - accuracy: 0.8560

Evaluating:
157/157 [==============================] - 1s 3ms/step - loss: 0.4142 - accuracy: 0.8480

Test accuracy: 84.8%, test loss: 0.414196


KeyboardInterrupt: 